Authors: Andreas Haupt, Jannis Kück, Alexander Quispe and Anzony Quispe, Vasilis Syrgkanis

# Testing the Convergence Hypothesis

In [1]:
!pip install multiprocess
!pip install pyreadr
!git clone https://github.com/maxhuppertz/hdmpy.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.7/411.7 kB 5.3 MB/s eta 0:00:00
Cloning into 'hdmpy'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 89 (delta 47), reused 63 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 28.88 KiB | 2.22 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [2]:
!pip install wget # for data loading

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=57633a0a84e854f12b0fb11a4ab14f528216558360f55ff92cb90f0ebefc97e9
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [3]:
import sys
import wget
import hdmpy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [4]:
from pyreadr import read_r

sys.path.insert(1, "./hdmpy")

## Introduction

We provide an additional empirical example of partialling-out with Lasso to estimate the regression coefficient $\beta_1$ in the high-dimensional linear regression model:
  $$
  Y = \beta_1 D +  \beta_2'W + \epsilon.
  $$
  
Specifically, we are interested in how the rates  at which economies of different countries grow ($Y$) are related to the initial wealth levels in each country ($D$) controlling for country's institutional, educational, and other similar characteristics ($W$).
  
The relationship is captured by $\beta_1$, the *speed of convergence/divergence*, which measures the speed at which poor countries catch up $(\beta_1< 0)$ or fall behind $(\beta_1> 0)$ rich countries, after controlling for $W$. Our inference question here is: do poor countries grow faster than rich countries, controlling for educational and other characteristics? In other words, is the speed of convergence negative: $ \beta_1 <0?$ This is the Convergence Hypothesis predicted by the Solow Growth Model. This is a structural economic model. Under some strong assumptions, that we won't state here, the predictive exercise we are doing here can be given causal interpretation.


The outcome $Y$ is the realized annual growth rate of a country's wealth  (Gross Domestic Product per capita). The target regressor ($D$) is the initial level of the country's wealth. The target parameter $\beta_1$ is the speed of convergence, which measures the speed at which poor countries catch up with rich countries. The controls ($W$) include measures of education levels, quality of institutions, trade openness, and political stability in the country.

## Data analysis


We consider the data set GrowthData which is included in the package *hdm*. First, let us load the data set to get familiar with the data.

In [5]:
# I downloaded the data that the author used
fname = "https://github.com/CausalAIBook/MetricsMLNotebooks/blob/main/data/GrowthData.rda?raw=true"
growth_read = read_r(wget.download(fname))

# Extracting the data frame from rdata_read
growth = growth_read['GrowthData']

We determine the dimension of our data set.

The sample contains $90$ countries and $63$ controls.

In [ ]:
growth.shape

Thus $p \approx 60$, $n=90$ and $p/n$ is not small. We expect the least squares method to provide a poor estimate of $\beta_1$.  We expect the method based on partialling-out with Lasso to provide a high quality estimate of $\beta_1$.
To check this hypothesis, we analyze the relation between the output variable $Y$ and the other country's characteristics by running a linear regression in the first step.

In [8]:
import statsmodels.api as sm

In [6]:
y = growth['Outcome']
X = growth.drop('Outcome', axis=1)

In [9]:
reg_ols = sm.OLS(y, X).fit(cov_type='HC1')


In [10]:
# output: estimated regression coefficient corresponding to the target regressor
est_ols = reg_ols.params['gdpsh465']
# output: HC1 std. error
std_ols = reg_ols.HC1_se['gdpsh465']
# output: 95% confidence interval
lower_ci, upper_ci = reg_ols.conf_int(alpha=0.05).loc['gdpsh465'].values

## Summarize OLS results

In [11]:
table = pd.DataFrame(columns=["Estimator", "Std. Error", "lower bound CI", "upper bound CI"])
table.loc['OLS'] = [est_ols, std_ols, lower_ci, upper_ci]
table

,Estimator,Std. Error,lower bound CI,upper bound CI
OLS,-0.009378,0.032421,-0.072922,0.054166


Least squares provides a rather noisy estimate (high standard error) of the
speed of convergence, and does not allow us to answer the question
about the convergence hypothesis since the confidence interval includes zero.

In contrast, we can use the partialling-out approach based on lasso regression ("Double Lasso").

In [43]:
y = growth['Outcome'].values
W = growth.drop(['Outcome',  'gdpsh465'], axis=1)
D = growth['gdpsh465'].values

In [44]:
res_y = sm.OLS(y, W).fit(cov_type='HC1').resid
res_D = sm.OLS(D,W).fit(cov_type='HC1').resid

r_y = pd.DataFrame(res_y, columns=['r_y'])
r_D = pd.DataFrame(res_D, columns=['r_D'])

In [45]:
reg_ols = sm.OLS(r_y, r_D).fit(cov_type='HC1')
# output: estimated regression coefficient corresponding to the target regressor
reg_ols.params

,0
r_D,-0.009378


## Method 1 - Lasso with Theoretical Setting of Penalty using HDMPY

While cross validation is commonly employed for choosing penalty parameters in Lasso, it can be very noisy and tends to choose relatively small penalty leading to some overfitting. For this reason, one should not use cross validation to choose tuning parameters unless sample splitting is employed. We illustrate the use of sample combined with cross validation in later chapters in the book. Since we are using the full sample here, it is much better (and theoretically valid) to use penalties that provably control overfitting, which is what we do here.

We report the results using cross validation at the end of this notebook for comparison. There, we observe overfitting for the prediction of the outcome.

In [ ]:
res_y = hdmpy.rlasso(W, y, post=False).est['residuals']
res_D = hdmpy.rlasso(W, D, post=False).est['residuals']

r_y = pd.DataFrame(res_y, columns=['r_y'])
r_D = pd.DataFrame(res_D, columns=['r_D'])

In [ ]:
# OLS regression
reg_ols = sm.OLS(r_y, r_D).fit(cov_type='HC1')

In [ ]:
reg_ols.params['r_D']

In [ ]:
est = reg_ols.params['r_D']
stderr = reg_ols.HC1_se['r_D']
lower_ci, upper_ci = reg_ols.conf_int(alpha=0.05).loc['r_D'].values
table.loc['Double Lasso'] = [est, stderr, lower_ci, upper_ci]
table

The least square method provides a rather noisy estimate of the speed of convergence. We can not answer the question if poor countries grow faster than rich countries. The least square method does not work when the ratio $p/n$ is large.

In sharp contrast, partialling-out via Lasso provides a more precise estimate. The Lasso based point estimate is $-5\%$ and the $95\%$ confidence interval for the (annual) rate of convergence $[-7.8\%,-2.2\%]$ only includes negative numbers. This empirical evidence does support the convergence hypothesis.

## Method 2: Using Cross-Validated Lasso

This section is for illustration purposes only. Given that we are using the full sample, cross validation *should not* be used for choosing tuning parameters here. Cross validation tends to (mildly) overfit, and this overfitting can lead to substantial problems when inference about parameters is the goal.

In [ ]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold

# Choose penalty based on KFold cross validation
# Given small sample size, we use an aggressive number of 20 folds
cv = KFold(n_splits=20, shuffle=True, random_state=123)
model_y = make_pipeline(StandardScaler(),
                        linear_model.LassoCV(n_alphas=10, cv=cv, max_iter=10000))

res_y = y - model_y.fit(W, y).predict(W)
r_y = pd.DataFrame(res_y, columns=['r_y'])

model_d = make_pipeline(StandardScaler(),
                        linear_model.LassoCV(n_alphas=10, cv=cv, max_iter=10000))
res_D = D - model_d.fit(W, D).predict(W)
r_D = pd.DataFrame(res_D, columns=['r_D'])

In [ ]:
# OLS regression
reg_ols = sm.OLS(r_y, r_D).fit(cov_type='HC1')

In [ ]:
est = reg_ols.params['r_D']
stderr = reg_ols.HC1_se['r_D']
lower_ci, upper_ci = reg_ols.conf_int(alpha=0.05).loc['r_D'].values
table.loc['Double Lasso-CV'] = [est, stderr, lower_ci, upper_ci]
table

We find that the outcome model chooses too small of a penalty based on cross-validation, leading to overfitting of the outcome and tiny outcome residuals. This leads to artificially small standard errors and a zero treatment effect. Theoretically driven penalty should be preferred for such small sample sizes.

In [ ]:
plt.plot(model_y[-1].alphas_, 1 - model_y[-1].mse_path_.mean(axis=1) / np.var(y))
plt.title('Outcome Lasso-CV Model: Out-of-sample R-squared as function of penalty level')
plt.show()
plt.title('Treatment Lasso-CV Model: Out-of-sample R-squared as function of penalty level')
plt.plot(model_d[-1].alphas_, 1 - model_d[-1].mse_path_.mean(axis=1) / np.var(D))
plt.show()

> *"El coeficiente de una variable en una regresión múltiple es exactamente el mismo que obtienes si primero 'parcias out' todas las demás variables..."*

---

### **Teorema de Frisch–Waugh–Lovell (versión OLS)**


$$
\hat{\gamma}_{YW} = \arg \min_{\gamma \in \mathbb{R}^p} \sum_{i} (Y_i - \gamma' W_i)^2
$$

$$
\hat{\gamma}_{DW} = \arg \min_{\gamma \in \mathbb{R}^p} \sum_{i} (D_i - \gamma' W_i)^2
$$

Donde:
- $Y_i$: es la variable objetivo
- $D_i$: regresor de interés
- $W_i$: Otros regresores





$$
\tilde{Y}_i = Y_i - \hat{\gamma}_{YW}' W_i
$$

$$
\tilde{D}_i = D_i - \hat{\gamma}_{DW}' W_i
$$


3. Corremos una regresión por OLS de $\tilde{Y}_i$ sobre $\tilde{D}_i$

El coeficiente estimado sobre $\tilde{D}_i$ es **exactamente igual** al estimado para $D_i$ en la regresión múltiple original de $Y_i$ sobre $D_i$ y $W_i$.

$\tilde{Y}_i$ sobre $\tilde{D}_i$:

$$
\tilde{Y}_i = \alpha + \theta \tilde{D}_i + u_i
$$

El estimador $\hat{\theta}$ deber ser **igual** al coeficiente $\hat{\beta}_D$ en la regresión múltiple original:

$$
Y_i = \alpha + \beta_D D_i + \beta_W' W_i + \varepsilon_i
$$
